#Imports

In [ ]:
!pip install spotipy

In [ ]:
%load_ext tensorboard
%tensorflow_version 2.X

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import spacy
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense,Conv2D,Conv1D, GlobalAveragePooling2D, MaxPooling2D,Flatten
from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

import re
import string
import requests
from bs4 import BeautifulSoup
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests

nlp = spacy.load("en_core_web_sm")

# Spotipy API Login
cid = 'b7bbbca585dc427badc487828339c228'
secret = '2ab78372f5d44d1ebb30df994100c245'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

from google.colab import drive
drive.mount('/content/drive')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.X`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Data

In [ ]:
# LOAD DATA
filepath='SpotifyFeatures.csv'
SpotifyFeatures_dataset = pd.read_csv(f'/content/drive/My Drive/{filepath}').drop_duplicates(subset=['track_id'])  # All Catergories including genres
df = SpotifyFeatures_dataset.copy()  # Duplicate orginial file for modifications
print("Shape:")
print(df.shape) 
print("\nColumn Headers:")
print(df.columns)
print("\nNaNs present?:")
print(df.isnull().sum())

# PREPROCESS
#Data Cleaning
numberValue = [0,1,2,3,4,5,6,7,8,9,10,11]
df['key'].unique()
df.replace(to_replace=['C#', 'F#', 'C', 'F', 'G', 'E', 'D#', 'G#', 'D', 'A#', 'A', 'B'], value=numberValue, inplace=True)
df.replace(to_replace=['Major','Minor'], value=[0,1], inplace=True)
df.replace(to_replace=['4/4', '5/4', '3/4', '1/4', '0/4'], value=[0,1,2,3,4], inplace=True)
df.dropna(inplace=True)
# df.drop_duplicates(subset=['track_id'], inplace=True)  # duplicates are actually dropped during read_csv process
#Dropping Data
# df.drop(['artist_name','track_id','track_name'],axis=1,inplace=True)  #If Need Be

Shape:
(176774, 18)

Column Headers:
Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

NaNs present?:
genre               0
artist_name         0
track_name          0
track_id            0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
dtype: int64


In [ ]:
# Export SpotifyFeatures df to .csv file
# SpotifyFeatures.to_csv('SpotifyFeatures_DF_Cleaned.csv')

# Data Exploration 

**put fancy graphs here**

In [ ]:
def top_artists_by_popularity(dataframe, lower_bound, upper_bound):
    

    reindex_order =  dataframe.groupby(['artist_name']).sum()['popularity'].sort_values(ascending = False)[lower_bound:upper_bound].index[::-1]     
    # order of index to make right values in right places
        
    total_value_of_popularity =  dataframe.groupby(['artist_name']).sum()['popularity'].sort_values(ascending = False)[
        lower_bound:upper_bound].reindex(reindex_order)
    
    total_songs = dataframe['artist_name'].value_counts().round(0).reindex(reindex_order)   
    
    df = pd.DataFrame({('Artist ') :total_value_of_popularity.index, 
                        ('Total Artist Popularity '): total_value_of_popularity.values,
                        ('Total Song Count '): total_songs.values}) 
  
    fig = px.bar(df, x = ('Total Artist Popularity ') , y = ('Artist '),
                    
                     hover_data = [('Total Artist Popularity '), ('Total Song Count ')],
                                       
                     color = ('Total Artist Popularity ') , title= f'Artists Popularity Ranked ({lower_bound +1},{upper_bound})',
               
                     height = 700  )

    return fig.show()

top_artists_by_popularity(df, 0, 20)

In [ ]:
#top artist by popularity
df.groupby("artist_name")["popularity"].sum().sort_values(ascending=False).head(20)

artist_name
Hans Zimmer                 19439
Wolfgang Amadeus Mozart     17785
Nobuo Uematsu               17714
Giuseppe Verdi              17495
Johann Sebastian Bach       16508
John Williams               15585
Frédéric Chopin             15193
Giacomo Puccini             14758
Ludwig van Beethoven        14597
Bob Marley & The Wailers    14520
Max Richter                 13353
Drake                       11048
Eminem                      10692
Hillsong Worship             9883
Howard Shore                 9787
Ramin Djawadi                9774
Randy Newman                 9730
Georges Bizet                9701
Richard Wagner               9495
The Beatles                  8965
Name: popularity, dtype: int64

In [ ]:
def top_n_artists_by_song_count(dataframe, lower_bound, upper_bound):
    
    reindex_order = dataframe['artist_name'].value_counts()[lower_bound:upper_bound].index[::-1]     
    # order of index to make right values in right places
        
    total_value_of_songs = dataframe['artist_name'].value_counts()[lower_bound:upper_bound].reindex(reindex_order)
    
    total_value_of_popularity =  dataframe.groupby(['artist_name']).sum()['popularity'].reindex(reindex_order)   # so we can see their popularity too
    
    df = pd.DataFrame({('Artist ') :total_value_of_songs.index, 
                        ('Total Song Count '): total_value_of_songs.values,
                        ('Total Artist Popularity '): total_value_of_popularity.values}) 
  
    fig = px.bar(df, x = ('Total Song Count ') , y = ('Artist '),
                    
                     hover_data = [('Total Song Count '), ('Total Artist Popularity ')],
                                       
                     color = ('Total Song Count ') , title= f'Artists Song Count Ranked ({lower_bound+1},{upper_bound}) ',
               
                     height = 700  )

    return fig.show()

top_n_artists_by_song_count(df,0,20)

In [ ]:
def rank_artist_similarity(data, artist, genre_parameter):

    artist_data = data[data.artist_name == artist]
    artist_genres = artist_data.genre#set(artist_data.genre)
    similarity_data = data.drop(artist_data.index)
    similarity_data.genres = similarity_data.genre.apply(lambda genres: list(set(genres).intersection(artist_genres)))
    similarity_lengths = similarity_data.genres.str.len()
    similarity_data = similarity_data.reindex(similarity_lengths[similarity_lengths >= genre_parameter].sort_values(ascending=False).index)
    similarity_data.rename(columns={'artists': f'Similar Artists to {artist}', 'genres': 'Similar Genres', 'popularity': 'Artist Popularity'}, inplace=True)
    
    return similarity_data

rank_artist_similarity(df, 'Eminem', 3)

# FIX ISSUE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



,genre,artist_name,track_name,track_id,Artist Popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


#Encode Categorical Columns (new df)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoded_df = df. copy()


genre_encoder = LabelEncoder()
encoded_df['genre'] = genre_encoder.fit_transform(encoded_df['genre'])

artist_encoder = LabelEncoder()
encoded_df['artist_name'] = artist_encoder.fit_transform(encoded_df['artist_name'])

song_encoder = LabelEncoder()
encoded_df['track_name'] = encoded_df['track_name'].astype(str)
encoded_df['track_name'] = song_encoder.fit_transform(encoded_df['track_name'])  #Raised Error: Needs to be str or int
encoded_df

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,15,5181,18317,0BRjO6ga9RKCKjfDqeFgWV,0,0.6110,0.389,99373,0.910,0.000000,0,0.3460,-1.828,0,0.0525,166.969,0,0.814
1,15,8254,92822,0BjC1NfoEOOusryehmNudP,1,0.2460,0.590,137373,0.737,0.000000,1,0.1510,-5.559,1,0.0868,174.003,0,0.816
2,15,6451,32139,0CoSDzoNIKCRs124s9uTVy,3,0.9520,0.663,170267,0.131,0.000000,2,0.1030,-13.879,1,0.0362,99.488,1,0.368
3,15,5181,30946,0Gc6TVm52BwZD07Ki6tIvf,0,0.7030,0.240,152427,0.326,0.000000,0,0.0985,-12.178,0,0.0395,171.758,0,0.227
4,15,4074,90654,0IuslXpMROHdEPvSl1fTQK,4,0.9500,0.331,82625,0.225,0.123000,3,0.2020,-21.150,0,0.0456,140.576,0,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232716,24,6276,97939,1U0OMWvR89Cm20vCNar50f,39,0.2310,0.736,222667,0.701,0.000000,9,0.2030,-4.345,1,0.1000,99.991,0,0.770
232717,24,1259,90986,2gGqKJWfWbToha2YmDxnnj,43,0.1040,0.802,201173,0.516,0.000485,8,0.1050,-9.014,0,0.2130,175.666,0,0.482
232719,24,1633,56686,2iZf3EUedz9MPqbAvXdpdA,32,0.5660,0.423,144667,0.337,0.000000,9,0.2760,-13.092,1,0.0436,80.023,0,0.497
232721,24,6508,17928,1qWZdkBl4UVPj9lK6HuuFM,38,0.0329,0.785,282447,0.683,0.000880,5,0.2370,-6.944,1,0.0337,113.830,0,0.969


Are all of the columns encoded numerically??? (minus the track ID)

In [ ]:
# encoded_df.info()

In [ ]:
# I think what is going on here is the 'track_id' col is being moved to the beginning of df
# In preparation for the forthcoming knn slicing function...
col_name = "track_id"
first_col = encoded_df.pop(col_name)
encoded_df.insert(0, col_name, first_col)

col_name = "track_name"
first_col = encoded_df.pop(col_name)
encoded_df.insert(1, col_name, first_col)

#Train, Test Split w/Target: ID

In [ ]:
traintest_df = encoded_df.copy()  # Make copy just in case

In [ ]:
traintest_df  # Double-Check everything is up to par

,track_id,track_name,genre,artist_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0BRjO6ga9RKCKjfDqeFgWV,18317,15,5181,0,0.6110,0.389,99373,0.910,0.000000,0,0.3460,-1.828,0,0.0525,166.969,0,0.814
1,0BjC1NfoEOOusryehmNudP,92822,15,8254,1,0.2460,0.590,137373,0.737,0.000000,1,0.1510,-5.559,1,0.0868,174.003,0,0.816
2,0CoSDzoNIKCRs124s9uTVy,32139,15,6451,3,0.9520,0.663,170267,0.131,0.000000,2,0.1030,-13.879,1,0.0362,99.488,1,0.368
3,0Gc6TVm52BwZD07Ki6tIvf,30946,15,5181,0,0.7030,0.240,152427,0.326,0.000000,0,0.0985,-12.178,0,0.0395,171.758,0,0.227
4,0IuslXpMROHdEPvSl1fTQK,90654,15,4074,4,0.9500,0.331,82625,0.225,0.123000,3,0.2020,-21.150,0,0.0456,140.576,0,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232716,1U0OMWvR89Cm20vCNar50f,97939,24,6276,39,0.2310,0.736,222667,0.701,0.000000,9,0.2030,-4.345,1,0.1000,99.991,0,0.770
232717,2gGqKJWfWbToha2YmDxnnj,90986,24,1259,43,0.1040,0.802,201173,0.516,0.000485,8,0.1050,-9.014,0,0.2130,175.666,0,0.482
232719,2iZf3EUedz9MPqbAvXdpdA,56686,24,1633,32,0.5660,0.423,144667,0.337,0.000000,9,0.2760,-13.092,1,0.0436,80.023,0,0.497
232721,1qWZdkBl4UVPj9lK6HuuFM,17928,24,6508,38,0.0329,0.785,282447,0.683,0.000880,5,0.2370,-6.944,1,0.0337,113.830,0,0.969


In [ ]:
from sklearn.model_selection import train_test_split

X = traintest_df[['genre', 'artist_name', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence']]

y = traintest_df['track_name']


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, test_size=0.15)
print(
    f"""
    X Train: {X_train.shape, type(X_train)}, 
    y train: {y_train.shape, type(y_train)},
    X Test: {X_test.shape, type(X_test)},
    y test: {y_test.shape, type(y_test)}
    """
)


    X Train: ((150257, 16), <class 'pandas.core.frame.DataFrame'>), 
    y train: ((150257,), <class 'pandas.core.series.Series'>),
    X Test: ((26517, 16), <class 'pandas.core.frame.DataFrame'>),
    y test: ((26517,), <class 'pandas.core.series.Series'>)
    


In [ ]:
# unique_class_labels =len(np.unique(y))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis, NearestNeighbors
from sklearn import metrics
from sklearn.pipeline import Pipeline

# WARNING
nca = NeighborhoodComponentsAnalysis(random_state=42)
nn_classifier = NearestNeighbors(n_neighbors=15, n_jobs=-1)
nca_pipe = Pipeline([
                     ('nca', nca), 
                     ('nn', nn_classifier)
          ])
# nca_pipe.fit(X_train, y_train)  # Runtime Crash!??...
# print(f'NCA Pipeline Score: {nca_pipe.score(X_test, y_test)}')

# knn_nca_y_pred = nca_pipe.predict(X_test)
# print(f'{metrics.accuracy_score("Actual:",y_test, "Predicted:", knn_nca_y_pred)}')

In [ ]:
nca_pipe

Pipeline(memory=None,
         steps=[('nca',
                 NeighborhoodComponentsAnalysis(callback=None, init='auto',
                                                max_iter=50, n_components=None,
                                                random_state=42, tol=1e-05,
                                                verbose=0, warm_start=False)),
                ('nn',
                 NearestNeighbors(algorithm='auto', leaf_size=30,
                                  metric='minkowski', metric_params=None,
                                  n_jobs=-1, n_neighbors=15, p=2,
                                  radius=1.0))],
         verbose=False)

In [ ]:
# Nearest Neighbors fit then predict 
from flask import jsonify
nneighbors = NearestNeighbors(n_neighbors=15, n_jobs=-1)
nneighbors.fit(encoded_df[encoded_df.columns[2:]], y)  #fit all columns w/ possible slice that will be used in algorithm. No indeces or track_id


def predict_tracks(data, track_id):
    id_index = encoded_df.index[encoded_df['track_id'] == track_id]
    # print(f"id: {id}")
    series = encoded_df.iloc[id_index, 2:].to_numpy()
    # print(f"series: {series}")
    nneighbors_ = nneighbors.kneighbors(series)
    new_observations = nneighbors_[1][0][6:26]
    encoded_predict = [encoded_df.track_id.iloc[i] for i in new_observations]
    print(f"{len(encoded_predict)} unique json Spotify track ids were returned. The converted song titles are the following:\n")
    json_output = df[df['track_id'].isin(encoded_predict)].track_name.values
 
    # convert dictionary string to dictionary 
    # return print({'track_id': json_output})
    return  json_output, encoded_predict

In [ ]:
# # Choose randon track id by .loc[] column
# df.iloc[10000]

In [ ]:
test_song='3ipzZgH1766XXRamVwk1Iw'
# Call Function
predict_tracks(encoded_df, test_song)

9 unique json Spotify track ids were returned. The converted song titles are the following:

{'track_id': array(['Young And In Love', 'Amargo Adiós', 'I Am Above', 'Oil and Water',
       'SPEED TRAP', 'I Am Above', 'Only Wanna Be with You',
       'The King of the Golden Hall', 'Envidia'], dtype=object)}


(array(['Young And In Love', 'Amargo Adiós', 'I Am Above', 'Oil and Water',
        'SPEED TRAP', 'I Am Above', 'Only Wanna Be with You',
        'The King of the Golden Hall', 'Envidia'], dtype=object),
 ['1OFKUn2VLafrHj7ybnap0Q',
  '1rnXgo3fhkivqzp40F3MQG',
  '2b69WEj0c5b6oj8MXcWZJS',
  '2O8AQadTJzQ3wdhSNRHdYg',
  '4bCNuJYfnWglrBsd9s9LDE',
  '1V7sPJKrG9BkVsAoezgQuB',
  '4mQwxVqjcHdUEfwSWEOopx',
  '6qgpBezjMvybos30jStS2Z',
  '0Ov4x4mro5sTaBlzlDuXYg'],
 None)

In [ ]:
next_song = df[df['track_id'].isin(encoded_predict)].track_name.values

In [ ]:
next_song

In [ ]:
df

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.6110,0.389,99373,0.910,0.000000,0,0.3460,-1.828,0,0.0525,166.969,0,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.2460,0.590,137373,0.737,0.000000,1,0.1510,-5.559,1,0.0868,174.003,0,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.9520,0.663,170267,0.131,0.000000,2,0.1030,-13.879,1,0.0362,99.488,1,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.7030,0.240,152427,0.326,0.000000,0,0.0985,-12.178,0,0.0395,171.758,0,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.9500,0.331,82625,0.225,0.123000,3,0.2020,-21.150,0,0.0456,140.576,0,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232716,Soul,John Legend,Quickly (feat. Brandy),1U0OMWvR89Cm20vCNar50f,39,0.2310,0.736,222667,0.701,0.000000,9,0.2030,-4.345,1,0.1000,99.991,0,0.770
232717,Soul,Belly,P.O.P.,2gGqKJWfWbToha2YmDxnnj,43,0.1040,0.802,201173,0.516,0.000485,8,0.1050,-9.014,0,0.2130,175.666,0,0.482
232719,Soul,"Bobby ""Blue"" Bland",I'll Take Care Of You - Single Version,2iZf3EUedz9MPqbAvXdpdA,32,0.5660,0.423,144667,0.337,0.000000,9,0.2760,-13.092,1,0.0436,80.023,0,0.497
232721,Soul,Jr Thomas & The Volcanos,Burning Fire,1qWZdkBl4UVPj9lK6HuuFM,38,0.0329,0.785,282447,0.683,0.000880,5,0.2370,-6.944,1,0.0337,113.830,0,0.969


In [ ]:
#Goal: To take the list of recommendations returned and find the best next song choice. 
# Create a dataframe using list of

In [ ]:
# Use a model of your choice to determine the next single song to play

In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer(stop_words='english',
        ngram_range=(1, 3),
        max_features=5000,
    )
nn = NearestNeighbors(n_neighbors=15, n_jobs=-1)
skf = StratifiedKFold(n_splits=2)
pipeline = Pipeline([
    ('vect', tfidf),
    ('clf', nn)
])
param_grid = {
    'vect__stop_words': [None],
    'vect__ngram_range': [(1,2), (1,3)],
    'vect__min_df': (0, 0.15),
    'vect__max_df': (0.55, 1.0),
}
rndm_cv = RandomizedSearchCV(
                        estimator=pipeline, 
                        param_distributions=param_grid, 
                        cv=skf, 
                        n_jobs=-1, 
                        verbose=10, 
                        return_train_score=True
    )

rndm_cv.fit(listofsongs, final_strains['name'])

NameError: ignored

In [ ]:
test_song = [test_song]
rndm_cv.predict(exatest_songmple2)

#Webscraping

In [ ]:
import bs4 as bs
import urllib.request

source = urllib.request.urlopen('https://pythonprogramming.net/parsememcparseface/').read()

In [ ]:
df.head(0)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


In [ ]:
df.iloc[0]

genre                                     Movie
artist_name                      Henri Salvador
track_name          C'est beau de faire un Show
track_id                 0BRjO6ga9RKCKjfDqeFgWV
popularity                                    0
acousticness                              0.611
danceability                              0.389
duration_ms                               99373
energy                                     0.91
instrumentalness                              0
key                                           0
liveness                                  0.346
loudness                                 -1.828
mode                                          0
speechiness                              0.0525
tempo                                   166.969
time_signature                                0
valence                                   0.814
Name: 0, dtype: object

In [ ]:
# Pickle Final Model
import joblib
filename = "NN_Model_Final.joblib"
item_to_save = nneighbors
joblib.dump(item_to_save, filename, compress=2, protocol=None, cache_size=None)

In [ ]:
dump(rndm_cv, 'Next_Single_to_Play_Model.joblib', compress=True)

# Load Pickle Model

In [ ]:
filename = 'NN_Model_Final.joblib'
NNeighbors_Model = joblib.load(filename)
# filename = "NN_Model_Final.joblib"
# with open(filename, 'rb') as fo:  # doctest: +ELLIPSIS
#       NNeighbors_Model = joblib.load(fo)

In [ ]:
# # Call Function
# test_song='2iZf3EUedz9MPqbAvXdpdA'
# predict_tracks(encoded_df, test_song)